# 2. Khám phá dữ liệu

## 2.1. Cấu trúc của tập dữ liệu

Dữ liệu trên gồm một bảng. Trong bảng gồm 7 thuộc tính:
- **country_or_area**: Quốc gia hoặc khu vực được khảo sát
- **commodity_transaction**: Hàng hóa và mục đích mục đích giao dịch của loại hàng đó
- **year**: Năm được lấy số liệu
- **unit**: Đơn vị đo của loại năng lượng
- **quantity**: Số lượng (tính theo đơn vị đo)
- **category**: phân loại năng lượng

## 2.2. Đọc dữ liệu, đếm số cột và số dòng

Import các thư viện cần thiết

In [1]:
import numpy as np
import pandas as pd

Đọc file **'Europe.csv'** và lưu vào dataframe **'europe_df'**

In [2]:
europe_df = pd.read_csv('Europe.csv')
europe_df.head()

,country_or_area,commodity_transaction,year,unit,quantity,quantity_footnotes,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,NaN,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,NaN,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,NaN,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,NaN,additives_and_oxygenates


Tính số dòng và số cột lưu vào biến num_rows và num_cols

In [3]:
num_rows, num_cols = europe_df.shape
print(f'Số dòng: {num_rows}')
print(f'Số cột: {num_cols}')

Số dòng: 382755
Số cột: 7


## 2.3. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Ở dataset này, mỗi dòng sẽ là lượng năng lượng được tiêu thụ cũng như trao đổi của các nước châu Âu. Mỗi dòng sẽ là lượng năng lượng đó của một nước trong một năm. Một nước sẽ có nhiều loại hình năng lượng khác nhau. Nhìn chung thì ý nghĩa của các dòng sẽ là tương tự nhau.

## 2.4. Dữ liệu có bị lặp hay không?

Biến **is_duplicated** sẽ trả về **True** nếu bị lặp hoặc **False** nếu không có dòng nào trùng nhau

In [4]:
is_duplicated = europe_df.duplicated().any()
is_duplicated

False

Như vậy, không có dòng nào bị trùng lặp với nhau cả

## 2.5. Mỗi cột có kiểu giá trị gì?

In [5]:
col_dtype = europe_df.dtypes
col_dtype

country_or_area           object
commodity_transaction     object
year                       int64
unit                      object
quantity                 float64
quantity_footnotes       float64
category                  object
dtype: object

### Nhận xét:
Nhìn chung thì kiểu dữ liệu của các cột đã tương đối phù hợp

## 2.6. Sự phân bố dữ liệu

### Ta sẽ kiểm tra dữ liệu bị thiếu

In [6]:
missing = pd.concat([europe_df.isnull().sum(), 100 * europe_df.isnull().mean()], axis=1)
missing.columns=['Count', '%']
missing.sort_values(by='Count', ascending=False)

,Count,%
quantity_footnotes,367612,96.043683
country_or_area,0,0.000000
commodity_transaction,0,0.000000
year,0,0.000000
unit,0,0.000000
quantity,0,0.000000
category,0,0.000000


Các giá trị phân biệt của cột quantity_footnotes

In [7]:
europe_df['quantity_footnotes'].unique()

array([nan,  1.])

#### Nhận xét:
- Như ta đã thấy, cột **quantity_footnotes** chỉ có giá trị 1 và còn thiếu nhiều. Và nó cũng không cho ta thêm nhiều dữ liệu cho quá trình phân tích
- Vì thế ta sẽ loại bỏ cột này đi để tiếp tục quá trình phân tích. 

In [8]:
df1 = europe_df.drop(['quantity_footnotes'], axis=1)
df1.columns

Index(['country_or_area', 'commodity_transaction', 'year', 'unit', 'quantity',
       'category'],
      dtype='object')

### Sự phân bố dữ liệu dạng số

Ta thấy có hai cột dữ liệu dạng số là **quantity** và **year**
Ở đây ta thấy tuy cùng là số ở cột **quantity** nhưng do có nhiều loại hình năng lượng khác nhau nên đơn vị đo cũng khác nhau. Để so sánh chính xác số lượng thì ta sẽ chia các dạng năng lượng theo cột **unit** để dễ dàng theo dõi
Ta sẽ thống kê các chỉ số sau cho dữ liệu số:
- Giá trị nhỏ nhất (min)
- Giá trị lớn nhất (max)
- Phân vị 25
- Phân vị 50
- Phân vị 75

Sự phân bố của năm (cột **year**)

In [9]:
num_df = europe_df.copy()
year_df = num_df['year']
year_df = year_df.to_numpy()
titles=['min', 'lower_quartile', 'median', 'upper_quartile', 'max']
temp = []
temp.append(np.nanmin(year_df))
temp.append(np.nanpercentile(year_df,25))
temp.append(np.nanpercentile(year_df,50))
temp.append(np.nanpercentile(year_df,70))
temp.append(np.nanmax(year_df))
year_dict = {}
year_dict['year'] = temp
year_dict['titles'] = titles
pd.DataFrame(year_dict).set_index('titles')

,year
titles,
min,1990.0
lower_quartile,1997.0
median,2003.0
upper_quartile,2008.0
max,2014.0


#### Nhận xét:
- Như ta đã thấy, dữ liệu được thu thập từ năm 1990 đến năm 2014, là 24 năm
- Trung bình năm là 2003, đầu thập niên 2000, đây là khoảng thời gian chuyển giao quan trọng và có nhiều thông tin để khai thác

Sự phân bố dữ liệu ở cột **quantity**, ở đây ta sẽ thống kê theo từng đơn vị đo

In [10]:
num_df = num_df[['quantity','unit']]
num_column = list(num_df.columns)
unit_set = list(set(num_df['unit'].tolist()))
columns =  unit_set
quantity_val = []
quantity_dict = {}
for cate in columns:
    temp = []
    unit_cate = num_df[num_df['unit'] == cate]
    arrays = unit_cate['quantity'].to_numpy()
    temp.append(np.nanmin(arrays,axis=0).round(3))
    temp.append(np.nanpercentile(arrays,25,axis=0).round(3))
    temp.append(np.nanpercentile(arrays,50,axis=0).round(3))
    temp.append(np.nanpercentile(arrays,70,axis=0).round(3))
    temp.append(np.nanmax(arrays,axis=0).round(3))
    quantity_val.append(temp)

for col in range(len(columns)):
    quantity_dict[columns[col]] = quantity_val[col]
    quantity_dict['titles']=titles   
    
pd.DataFrame(quantity_dict).set_index('titles')


,"Cubic metres, thousand","Metric tons, thousand","Kilowatts, thousand","Kilowatt-hours, million",Terajoules,Metric Tons
titles,,,,,,
min,-1779.967,-1.928800e+04,0.00,0.0,-4.335110e+05,0.0
lower_quartile,13.136,6.000000e+00,50.00,345.0,3.450000e+02,24.0
median,258.566,6.800000e+01,1216.00,2408.0,3.036000e+03,120.0
upper_quartile,1469.787,3.490000e+02,4270.05,9767.2,1.287940e+04,452.0
max,49292.282,2.727915e+09,259020.00,2896000.0,1.879251e+09,432000.0


#### Nhận xét:
- Như ta đã thấy, sự chênh lệch của gái trị min và max của mỗi cột là khá lớn
- Ở đây có cả những giá trị âm, chứng tỏ quá trình trao đổi không phải lúc nào cũng đem về kết quả dương

Ta sẽ tiếp tục tìm hiểu xem nhưng giá trị âm này tại sao lại có mặt
Ở đây ta sẽ xem loại hàng hóa và mục đích của những hàng có **quantity** âm

In [11]:
commo_transac = []
for i in range(100000):
    if europe_df.iloc[i]['quantity'] < 0:
        commo_transac.append(europe_df.iloc[i]['commodity_transaction'])
        commo_type = set(commo_transac)
print(commo_type)

{'Aviation gasoline - Total energy supply', 'Aviation gasoline - Stock changes', 'Additives and Oxygenates - Stock changes', 'Biodiesel - Stock changes', 'Charcoal - Stock changes', 'Conventional crude oil - stock changes', 'Brown coal briquettes - stock changes', 'Coal Tar - Stock changes', 'Charcoal - Total energy supply', 'Biodiesel - Net transfers', 'Coking coal - stock changes', 'Biogasoline - stock changes', 'Bitumen - Stock changes', 'Ethane - Stock changes', 'Additives and Oxygenates - Total energy supply', 'Coke Oven Gas - Stock changes', 'Conventional crude oil - transfers and recycled products', 'Brown coal - changes in stocks', 'Aviation gasoline - Transfers and recycled products', 'Bitumen - Transfers and recycled products', 'Anthracite - stock changes', 'Biogases - Stock changes'}


#### Nhận  xét
- Như đã thấy, chủ yếu những giá trị **quantity** âm đến từ những hoạt động trao đổi hàng hóa - việc trao đổi hàng hóa có thể lỗ và trả về âm. Một số ở lượng năng lượng cung cấp. Do lượng cung cấp thấp hơn như cầu sử dụng nên mang giá trị âm.

### Sự phân bố dữ liệu không phải dạng số

Ở đây ta sẽ xem xét những giá trị:
- Số các giá trị khác nhau của từng cột
- Giá trị xuất hiện nhiều lần nhất của một cột

In [13]:
nonnum_df=europe_df.select_dtypes('object')
nonnum_titles = ['seperated_num','most_frequent','max_ratio']
col_names = nonnum_df.columns
values_list=[]
dic={}
for i in col_names:
    cate = {}
    num = 0
    temp = []
    df=nonnum_df.copy()[[i]]
    count = len(df)
    temp.append(count)
    diff = set(df[i])
    cate = dict.fromkeys(diff,0)
    for key in df[i]:
        cate[key] = cate[key] + 1
    max_val = max(cate,key = cate.get)
    temp.append(max_val)
    ratio = cate[max_val]/num_rows*100
    temp.append(ratio)
    values_list.append(temp)
    
for col in range(len(col_names)):
    dic[col_names[col]] = values_list[col]
    dic['titles']=nonnum_titles 
    
pd.DataFrame(dic).set_index('titles')

,country_or_area,commodity_transaction,unit,category
titles,,,,
seperated_num,382755,382755,382755,382755
most_frequent,Germany,From combustible fuels – Main activity,"Metric tons, thousand",total_electricity
max_ratio,5.335528,0.328931,61.494951,8.769578


#### Nhận xét:
- Như ta thấy ở trên, loại hình năng lượng được xuất hiện phổ biến nhất chính là điện năng, điều này hoàn toàn hợp lý ở thời hiện đại này

## Tiền xử lý dữ liệu

### Như ta đã phân tích ở trên, cột quantity_footnotes là không cần thiết, vì vậy ta sẽ loại bỏ cột đó ra khỏi dataset

In [16]:
new_df = europe_df.drop(['quantity_footnotes'], axis=1)
new_df.to_csv("Europe_new.csv",index=False)
new_df

,country_or_area,commodity_transaction,year,unit,quantity,category
0,Austria,Additives and Oxygenates - Exports,1996,"Metric tons, thousand",5.0,additives_and_oxygenates
1,Austria,Additives and Oxygenates - Exports,1995,"Metric tons, thousand",17.0,additives_and_oxygenates
2,Belgium,Additives and Oxygenates - Exports,2014,"Metric tons, thousand",0.0,additives_and_oxygenates
3,Belgium,Additives and Oxygenates - Exports,2013,"Metric tons, thousand",0.0,additives_and_oxygenates
4,Belgium,Additives and Oxygenates - Exports,2012,"Metric tons, thousand",35.0,additives_and_oxygenates
...,...,...,...,...,...,...
382750,United Kingdom,Electricity - total wind production,1994,"Kilowatt-hours, million",342.0,wind_electricity
382751,United Kingdom,Electricity - total wind production,1993,"Kilowatt-hours, million",218.0,wind_electricity
382752,United Kingdom,Electricity - total wind production,1992,"Kilowatt-hours, million",40.0,wind_electricity
382753,United Kingdom,Electricity - total wind production,1991,"Kilowatt-hours, million",11.0,wind_electricity
